In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import tensorflow as tf
import keras
import keras.layers as L
import math
from keras.utils import Sequence
from keras.preprocessing import image
from random import shuffle
from sklearn.model_selection import train_test_split
import os

In [ ]:
train_labels = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
sample_submission = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

In [ ]:
train_labels.head()

In [ ]:
train_labels.shape

In [ ]:
sns.countplot(x='target',data=train_labels,palette='Set2')

In [ ]:
train_labels['id'][:5]

In [ ]:
#numpy Files are extracted
path = list(train_labels['id'])  
for i in range(len(path)):
    path[i] = '../input/g2net-gravitational-wave-detection/train/' +path[i][0]+'/'+path[i][1]+'/'+path[i][2]+'/' + path[i] + '.npy'

In [ ]:
path[0:5]

In [ ]:
def id2path(idx,is_train=True):
    path = '../input/g2net-gravitational-wave-detection'
    if is_train:
        path += '/train/'+idx[0]+'/'+idx[1]+'/'+idx[2]+'/'+idx+'.npy'
    else:
        path += '/test/'+idx[0]+'/'+idx[1]+'/'+idx[2]+'/'+idx+'.npy'
    return path

In [ ]:
!pip install -q nnAudio

In [ ]:
import torch
from nnAudio.Spectrogram import CQT1992v2
def increase_dimension(idx,is_train,transform=CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)): # in order to use efficientnet we need 3 dimension images
    waves = np.load(id2path(idx,is_train))
    waves = np.hstack(waves)
    waves = waves / np.max(waves)
    waves = torch.from_numpy(waves).float()
    image = transform(waves)
    image = np.array(image)
    image = np.transpose(image,(1,2,0))
    return image

In [ ]:
example = np.load(path[0])

In [ ]:
fig,a =  plt.subplots(3,1)
a[0].plot(example[1],color='green')
a[1].plot(example[1],color='red')
a[2].plot(example[1],color='yellow')
fig.suptitle('Target 1', fontsize=16)
plt.show()
plt.imshow(increase_dimension(train_labels['id'][0],is_train=True))
plt.show()

In [ ]:
example = np.load(path[10])
fig,a =  plt.subplots(3,1)
a[0].plot(example[1],color='green')
a[1].plot(example[1],color='red')
a[2].plot(example[1],color='yellow')
fig.suptitle('Target 0', fontsize=16)
plt.show()
plt.imshow(increase_dimension(train_labels['id'][1],is_train=True))
plt.show()

In [ ]:
example = np.load(path[20])
fig,a =  plt.subplots(3,1)
a[0].plot(example[1],color='green')
a[1].plot(example[1],color='red')
a[2].plot(example[1],color='yellow')
fig.suptitle('Target 0', fontsize=16)
plt.show()
plt.imshow(increase_dimension(train_labels['id'][1],is_train=True))
plt.show()

In [ ]:
example = np.load(path[5012])
fig,a =  plt.subplots(3,1)
a[0].plot(example[1],color='green')
a[1].plot(example[1],color='red')
a[2].plot(example[1],color='yellow')
fig.suptitle('Target 0', fontsize=16)
plt.show()
plt.imshow(increase_dimension(train_labels['id'][1],is_train=True))
plt.show()

In [ ]:
class Dataset(Sequence):
    def __init__(self,idx,y=None,batch_size=256,shuffle=True):
        self.idx = idx
        self.batch_size = batch_size
        self.shuffle = shuffle
        if y is not None:
            self.is_train=True
        else:
            self.is_train=False
        self.y = y
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
    def __getitem__(self,ids):
        batch_ids = self.idx[ids * self.batch_size:(ids + 1) * self.batch_size]
        if self.y is not None:
            batch_y = self.y[ids * self.batch_size: (ids + 1) * self.batch_size]
            
        list_x = np.array([increase_dimension(x,self.is_train) for x in batch_ids])
        batch_X = np.stack(list_x)
        if self.is_train:
            return batch_X, batch_y
        else:
            return batch_X
    
    def on_epoch_end(self):
        if self.shuffle and self.is_train:
            ids_y = list(zip(self.idx, self.y))
            shuffle(ids_y)
            self.idx, self.y = list(zip(*ids_y))

In [ ]:
train_idx =  train_labels['id'].values
y = train_labels['target'].values
test_idx = sample_submission['id'].values

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(train_idx,y,test_size=0.05,random_state=42,stratify=y)

In [ ]:
train_dataset = Dataset(x_train,y_train)
valid_dataset = Dataset(x_valid,y_valid)
test_dataset = Dataset(test_idx)

In [ ]:
x_train

In [ ]:
!pip install -U efficientnet

In [ ]:
import efficientnet.keras as efn

In [ ]:
model = tf.keras.Sequential([L.InputLayer(input_shape=(69,193,1)),L.Conv2D(3,3,activation='relu',padding='same'),efn.EfficientNetB0
                             (include_top=False,input_shape=(),weights='imagenet'),
        L.GlobalAveragePooling2D(),
        L.Dense(32,activation='relu'),
        L.Dense(1, activation='sigmoid')])

model.summary()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

In [ ]:
model.fit(train_dataset,epochs=1,validation_data=valid_dataset)

In [ ]:
model.save('./model_efn.h5')

In [ ]:
preds = model.predict(test_dataset)
preds = preds.reshape(-1)

In [ ]:
df = pd.DataFrame({'id':sample_submission['id'],'target':preds})

In [ ]:
df.to_csv('submission_efn.csv',index=False)

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
model_res = tf.keras.Sequential([L.InputLayer(input_shape=(69,193,1)),
                                 L.Conv2D(3,3,activation='relu',padding='same'),
                                 ResNet50(include_top=False,input_shape=(),weights='imagenet'),
        L.GlobalAveragePooling2D(),
        L.Dense(32,activation='relu'),
        L.Dense(1, activation='sigmoid')])

model_res.summary()
model_res.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

In [ ]:
train_dataset = Dataset(x_train,y_train)
valid_dataset = Dataset(x_valid,y_valid)
test_dataset = Dataset(test_idx)

In [ ]:
model_res.fit(train_dataset,epochs=1,validation_data=valid_dataset)

In [ ]:
model_res.save('./model_res.h5')

In [ ]:
preds = model_res.predict(test_dataset)
preds = preds.reshape(-1)

In [ ]:
df_res = pd.DataFrame({'id':sample_submission['id'],'target':preds})

In [ ]:
df_res.to_csv('submission_res.csv',index=False)

In [ ]:
from tensorflow.keras.applications import Xception

In [ ]:
model_xce = tf.keras.Sequential([L.InputLayer(input_shape=(69,193,1)),
                                 L.Conv2D(3,3,activation='relu',padding='same'),
                                 Xception(include_top=False,input_shape=(),weights='imagenet'),
        L.GlobalAveragePooling2D(),
        L.Dense(32,activation='relu'),
        L.Dense(1, activation='sigmoid')])

model_xce.summary()
model_xce.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

In [ ]:
train_dataset = Dataset(x_train,y_train)
valid_dataset = Dataset(x_valid,y_valid)
test_dataset = Dataset(test_idx)

In [ ]:
model_xce.fit(train_dataset,epochs=1,validation_data=valid_dataset)

In [ ]:
model_xce.save('./model_xce.h5')

In [ ]:
preds = model_xce.predict(test_dataset)
preds = preds.reshape(-1)

In [ ]:
df_xce = pd.DataFrame({'id':sample_submission['id'],'target':preds})

In [ ]:
df_xce.to_csv('submission_xce.csv',index=False)

In [ ]:
print('tb')

In [ ]:
# Ensemble Process

In [ ]:
def voting(a,b,c):
    if a==b:
        return a
    if b==c:
        return c
    if a==c:
        return a

In [ ]:
id1 = df['id']
target = []
for i in range(len(df['target'])):
    target.append(voting(df['target'][i],df_res['target'][i],df_xce['target'][i]))

    
finalsubmission = pd.DataFrame(columns = ['id','target'])

finalsubmission['id'] = id1
finalsubmission['target'] = target
finalsubmission.to_csv('submission.csv', index = False)